In [1]:
from rap_db import *
from rap_clean import*
from rap_viz import *

In [8]:
doom = art_load(['Doom'])['Doom']
chief = art_load(['Chief Keef'])['Chief Keef']

In [9]:
len(doom.uniq_art_verses)

112

In [10]:
len(chief.uniq_art_verses)

148

In [1]:
#this needs to look at a rap line and pull the white board metrics
#needs to be as fast and simple as possible as it will have to run on every training, testing, and validation data point
def make_line_metrics(rap_line):
    return

In [ ]:
#this takes all words and makes them into stems
#taken from rap clean, needs adjustments as we go
from nltk.stem import *
def split_on_stem(rap_line):
    stemmer = SnowballStemmer("english")
    all_words = re.sub("[^0-9a-zA-Z\']+", ' ', rap_line).split(' ')
    words_stm = [stemmer.stem(x.lower()) for x in all_words if stemmer.stem(x.lower()) not in stopwords.words('english')]
    self.all_stemmed_words = list(filter(None, words_stm))
    self.unique_stemmed_words = set(self.all_stemmed_words)

What you did
shell for line metrics

What you need to do next
Train models using two different training methadologies
1. text bag of words (simply look at words in textand classify using a naive bayes, random forest, SVM)
2. make a row for every line based on the whiteboarded lingustic measures (def use svm, maybe random forest, maybe KNN)

each artist should have a list of rap_line objects. Each line should have a word object dictionary, the preserved words as list of strings and then stemmed words/with unique stuff

Long term